In [1]:
import pyvisa
import numpy as np
import pandas as pd
import os
import csv
import time

In [1]:
from LCR import run, corrections, post_processing, frequency

ModuleNotFoundError: No module named 'LCR'

### Fil out the cell below with the information of the specific run


#### Define all the run variables in the cell below

In [11]:
# Experimental Conditions
function= 'ZTR'
volt= 0.01

# Frequency sweep
freq_list= frequency.freq_list(2E+5, 20, pts_per_decade=10)

# File information to use as the filename
sample = 'standard' 
temp =  '20C'
dist = '1mm'
# the save location. 
save_location = './EIS_Rheometer/'
comment = 'testing comment section'

### Run these cells to connect the instrument and interface with in using the code 

In [10]:
LCR = run.instrument_conenction()

The is/are 1 insturment(s) conencted to this device.

The selected instrument is:  Agilent Technologies,E4980A,MY46205278,A.02.20



### Then run all these cells without any additional change

In [14]:
EIS_measurements = run.measurement(LCR, function, freq_list, input_signal= 'voltage', amplitude= volt)

name = post_processing.gen_filename(sample, dist, temp)  

post_processing.write_file(freq_list, EIS_measurements, name, save_location, comment)

The file was correctly saved
